In [2]:
import pandas as pd
from lxml import html
import requests
import zipfile
import io
import os
from pathlib import Path
import csv

In [3]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Extracao_Demonstracoes'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [4]:
PARAMETROS 

{'caminho_arquivo_download': 'DADOS_DEMONSTRACOES/'}

In [5]:
PARAMETROS_GERAIS

{'pasta_raiz_dados': '/opt/DADOS_ANS5/',
 'arquivo_operadoras': 'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/Relatorio_cadop.csv',
 'arquivo_final_todas_demonstracoes': 'todas_demonstracoes_concatenadas.csv',
 'ano_inicio_analises': '2013'}

In [6]:
PASTA_DE_DADOS = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_download']

In [ ]:
page = requests.get('http://dados.gov.br/dataset/http-www-ans-gov-br-perfil-do-setor-dados-abertos-dados-abertos-disponiveis-n3')
tree = html.fromstring(page.content)

In [ ]:
link_resources_list = tree.xpath('//a[@class="resource-url-analytics"]')

In [ ]:
resources_list = []
for resource in link_resources_list:
    resources_list.append(resource.get("href"))

In [ ]:
"""
Inicia a extração do segundo elemento, pois o primeiro no html é o dicionário de dados
"""
for idx,link in enumerate(resources_list[1:]):
    print("Extraindo... ",(idx+1),"de",len(resources_list[1:]),link)
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(PASTA_DE_DADOS)

In [ ]:
lista_arquivos = os.listdir(PASTA_DE_DADOS)
lista_arquivos = [(PASTA_DE_DADOS+ "/" + x) for x in lista_arquivos if not str(x) == "nan"]

In [ ]:
"""
Chamada genérica para "puxar" todos os arquivos para dentro da pasta 'dados'
"""
for arquivo in lista_arquivos:
    caminho_absoluto = Path(arquivo).absolute()
    pasta_dados = Path(PASTA_DE_DADOS).absolute()
    if (os.path.isdir(caminho_absoluto)):
        for _arquivo in os.listdir(caminho_absoluto):
            _de = str(caminho_absoluto)+"/"+_arquivo
            _para = str(pasta_dados)+"/"+_arquivo
            os.rename(_de,_para)
        ""    
        caminho_absoluto.rmdir()
        

In [10]:
df_ajuste = pd.read_csv(PASTA_DE_DADOS+"2T2018.csv",sep=";",encoding="ISO-8859-1",dtype=str)
df_ajuste.dropna(inplace=True, how="all")


In [13]:
df_ajuste = df_ajuste[['DATA', 'REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO','VL_SALDO_FINAL']]

In [14]:
df_ajuste.to_csv(PASTA_DE_DADOS+"2T2018.csv",sep=";",encoding="ISO-8859-1",index=False)